## Claude 3.5 SonnetSolution

This solution is based on the XGBoost solution in the seperate file. Claude 3.5 Sonnet was then asked to generate a PyTorch version that performs equivalently. 

It's worth noting that the submission below doesn't perform quite as well as XGBoost, which is evidently similar to what is expected. 

In [1]:
import pandas as pd
import torch
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split


# Define the network
class HousePriceNet(nn.Module):
    def __init__(self, input_size=79):
        super().__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 1)
        self.dropout = nn.Dropout(0.2)
        self.batch_norm1 = nn.BatchNorm1d(256)
        self.batch_norm2 = nn.BatchNorm1d(128)
        self.batch_norm3 = nn.BatchNorm1d(64)

    def forward(self, x):
        x = self.batch_norm1(torch.relu(self.fc1(x)))
        x = self.dropout(x)
        x = self.batch_norm2(torch.relu(self.fc2(x)))
        x = self.dropout(x)
        x = self.batch_norm3(torch.relu(self.fc3(x)))
        x = self.fc4(x)
        return x


# Data loading and preprocessing (same as before)
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_concat = pd.concat([df_train, df_test])

# Preprocessing (similar to XGBoost version)
X = df_concat.drop(["SalePrice", "Id"], axis=1)
y = df_concat["SalePrice"]

categorical_columns = X.select_dtypes(include=["object"]).columns
numerical_columns = X.select_dtypes(include=["int64", "float64"]).columns

# Label encode categorical columns
label_encoders = {}
for col in categorical_columns:
    label_encoders[col] = LabelEncoder()
    X[col] = X[col].fillna("missing")
    X[col] = label_encoders[col].fit_transform(X[col])

# Handle numerical missing values
imputer = SimpleImputer(strategy="mean")
X[numerical_columns] = imputer.fit_transform(X[numerical_columns])

# Scale features and target
X_scaler = StandardScaler()
y_scaler = StandardScaler()

X = pd.DataFrame(X_scaler.fit_transform(X), columns=X.columns)
y = pd.Series(y_scaler.fit_transform(y.values.reshape(-1, 1)).ravel())

# Split back into train/test
X, X_test = X.iloc[: df_train.shape[0]], X.iloc[df_train.shape[0] :]
y, y_test = y.iloc[: df_train.shape[0]], y.iloc[df_train.shape[0] :]

# Create train/validation split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.FloatTensor(X_train.values)
y_train = torch.FloatTensor(y_train.values).reshape(-1, 1)
X_val = torch.FloatTensor(X_val.values)
y_val = torch.FloatTensor(y_val.values).reshape(-1, 1)
X_test = torch.FloatTensor(X_test.values)

# Training setup
model = HousePriceNet()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", factor=0.5, patience=5, verbose=True
)

# Training loop
n_epochs = 100
batch_size = 32
best_val_loss = float("inf")
patience = 10
patience_counter = 0

for epoch in range(n_epochs):
    model.train()
    # Mini-batch training
    for i in range(0, len(X_train), batch_size):
        batch_X = X_train[i : i + batch_size]
        batch_y = y_train[i : i + batch_size]

        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, y_val)

    # Print progress
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Val Loss: {val_loss.item():.4f}")

    # Learning rate scheduling
    scheduler.step(val_loss)

    # Early stopping
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "best_model.pth")
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch}")
            break

# Load best model and make predictions
model.load_state_dict(torch.load("best_model.pth"))
model.eval()

# Make predictions
with torch.no_grad():
    y_val_pred = model(X_val)
    y_test_pred = model(X_test)

# Convert predictions back to original scale
y_val_actual = y_scaler.inverse_transform(y_val.numpy())
y_val_pred_actual = y_scaler.inverse_transform(y_val_pred.numpy())
y_submission = y_scaler.inverse_transform(y_test_pred.numpy())

# Create submission file
df_submission = pd.DataFrame({"Id": df_test["Id"], "SalePrice": y_submission[:, 0]})
df_submission.to_csv("submission_torch.csv", index=False)

# Print validation results
df = pd.DataFrame(
    {"y_val_actual": y_val_actual[:, 0], "y_pred_actual": y_val_pred_actual[:, 0]}
)
print("\nValidation Results:")
print(df.head())

/Users/olivernormand/GitHub/veritas/.venv/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 0, Val Loss: 0.2342
Epoch 10, Val Loss: 0.1723
Epoch 20, Val Loss: 0.1375
Epoch 30, Val Loss: 0.1283
Early stopping at epoch 38

Validation Results:
   y_val_actual  y_pred_actual
0      154500.0  148498.890625
1      325000.0  321701.781250
2      115000.0  109539.359375
3      159000.0  173064.656250
4      315500.0  366053.593750


/var/folders/0q/j9hypd8x28q8j95_jd0b_mlm0000gn/T/ipykernel_9141/3618232270.py:126: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model